In [ ]:
import pandas as pd
import re

# ======================================================
# 1) 파일 업로드 및 불러오기
# ======================================================
file = "/content/지역별_유흥주점 정보.xlsx"

df = pd.read_excel(file)

# 공백 제거
df.columns = df.columns.str.strip()

print("불러온 컬럼:", df.columns.tolist())


# ======================================================
# 2) 주소에서 서울시 25개구 추출하는 함수
# ======================================================
seoul_gu_list = [
    "종로구", "중구", "용산구", "성동구", "광진구", "동대문구", "중랑구",
    "성북구", "강북구", "도봉구", "노원구", "은평구", "서대문구", "마포구",
    "양천구", "강서구", "구로구", "금천구", "영등포구", "동작구", "관악구",
    "서초구", "강남구", "송파구", "강동구"
]

gu_pattern = "(" + "|".join(seoul_gu_list) + ")"

def extract_gu(addr):
    if pd.isna(addr):
        return None
    match = re.search(gu_pattern, addr)
    return match.group(1) if match else None


# ======================================================
# 3) 주소 컬럼 처리 ('소재지전체주소' → 없으면 '도로명전체주소')
# ======================================================
addr_col = None
if "소재지전체주소" in df.columns:
    addr_col = "소재지전체주소"
elif "도로명전체주소" in df.columns:
    addr_col = "도로명전체주소"
else:
    raise ValueError("주소 컬럼이 없습니다. '소재지전체주소' 또는 '도로명전체주소' 필요")

df['구'] = df[addr_col].apply(extract_gu)

# 서울 구만 필터링
df = df[df['구'].notna()]
print("서울 지역 데이터 개수:", len(df))


# ======================================================
# 4) 인허가일자에서 연도 추출
# ======================================================
if '인허가일자' not in df.columns:
    raise ValueError("'인허가일자' 컬럼이 없습니다.")

# 문자열 날짜 또는 숫자 날짜 모두 처리
df['인허가일자'] = pd.to_datetime(df['인허가일자'], errors='coerce')

df['연도'] = df['인허가일자'].dt.year
df = df[df['연도'].notna()]  # 연도 없는 값 제거
df['연도'] = df['연도'].astype(int)


# ======================================================
# 5) 연도별 + 구별 등록된 유흥주점 수
# ======================================================
year_gu_count = df.groupby(['연도', '구']).size().reset_index(name="당해연도_등록수")


# ======================================================
# 6) 누적 합 계산 (총 유흥주점 수 = 저번년도 + 이번년도)
# ======================================================
year_gu_count = year_gu_count.sort_values(['구', '연도'])
year_gu_count['누적_유흥주점수'] = year_gu_count.groupby('구')['당해연도_등록수'].cumsum()


# ======================================================
# 7) CSV로 저장
# ======================================================
output_file = "/content/서울시_연도별_구별_유흥주점_누적집계.csv"
year_gu_count.to_csv(output_file, index=False, encoding='utf-8-sig')

output_file


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


불러온 컬럼: ['번호', '개방서비스명', '개방서비스아이디', '개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태구분코드', '영업상태명', '상세영업상태코드', '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '소재지전화', '소재지면적', '소재지우편번호', '소재지전체주소', '도로명전체주소', '도로명우편번호', '사업장명', '최종수정시점', '데이터갱신구분', '데이터갱신일자', '업태구분명', '좌표정보X(EPSG5174)', '좌표정보Y(EPSG5174)', '위생업태명', '남성종사자수', '여성종사자수', '영업장주변구분명', '등급구분명', '급수시설구분명', '총직원수', '본사직원수', '공장사무직직원수', '공장판매직직원수', '공장생산직직원수', '건물소유구분명', '보증액', '월세액', '다중이용업소여부', '시설총규모', '전통업소지정번호', '전통업소주된음식', '홈페이지']
서울 지역 데이터 개수: 1731


'/content/서울시_연도별_구별_유흥주점_누적집계.csv'